In [1]:
import os, sys
from osgeo import gdal

In [8]:
BASE_DIR = "/media/quothbonney/SRTM/"
ORI_DIR = os.path.join(BASE_DIR, "3601/")

In [10]:
ori_files = os.listdir(ORI_DIR)

In [11]:
3601 // 512

7

In [12]:
NEW_DIR = os.path.join(BASE_DIR, "512/")

In [18]:
import numpy as np

def write_subrasters(directory, raster_array, size):
  num_rasters = 3601 // size
  index = 0
  for i in range(num_rasters):
      for j in range(num_rasters):
        # Define the cropping window
        xoff, yoff, xsize, ysize = i * 512, j * 512, 512, 512 

        # Crop the raster array
        cropped_array = raster_array[yoff:yoff+ysize, xoff:xoff+xsize]
        unique, counts = np.unique(cropped_array, return_counts=True)
        if unique[0] == 0 and counts[0] > (0.6 * size*size):
           continue
        print(cropped_array)
        # Create a new raster file for the cropped array
        driver = gdal.GetDriverByName('GTiff')
        output_raster = driver.Create(os.path.join(NEW_DIR,  f"{directory}/{index}.tif"), xsize, ysize, 1, gdal.GDT_Float32)

        # Write the cropped array to the output raster
        output_raster.GetRasterBand(1).WriteArray(cropped_array)

        # Set the georeferencing information of the output raster
        output_raster.SetGeoTransform((input_raster.GetGeoTransform()[0]+xoff*input_raster.GetGeoTransform()[1],
                                      input_raster.GetGeoTransform()[1],
                                      0,
                                      input_raster.GetGeoTransform()[3]+yoff*input_raster.GetGeoTransform()[5],
                                      0,
                                      input_raster.GetGeoTransform()[5]))

        # Save and close the output raster file
        output_raster.FlushCache()
        output_raster = None
        index += 1

In [22]:
for f in ori_files:
    os.system(f"mkdir {NEW_DIR + f}")

In [54]:
gdal.Open("/media/quothbonney/SRTM/512/0.tif").ReadAsArray()

array([[ 0.,  0.,  0., ..., 15., 15., 15.],
       [ 0.,  0.,  0., ..., 17., 17., 16.],
       [ 0.,  0.,  0., ..., 19., 18., 18.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [23]:
# Open the input raster file
fname = os.path.join(ORI_DIR, ori_files[2])
for fname in ori_files:
  print(fname)
  input_raster = gdal.Open(ORI_DIR + fname)

  # Read the raster as a NumPy array
  raster_array = input_raster.ReadAsArray()

  write_subrasters(directory=fname, raster_array=raster_array, size=512)

n27_w081_1arc_v3.tif
[[21 19 18 ... 18 17 15]
 [20 18 18 ... 16 15 14]
 [19 18 18 ... 15 12 14]
 ...
 [23 21 21 ... 19 17 15]
 [22 19 19 ... 22 18 16]
 [22 20 21 ... 21 19 18]]
[[23 22 22 ... 20 20 21]
 [25 24 23 ... 21 22 23]
 [25 24 23 ... 21 22 22]
 ...
 [20 21 22 ... 31 29 27]
 [22 22 22 ... 31 31 31]
 [22 21 21 ... 30 31 31]]
[[21 21 21 ... 30 31 31]
 [22 22 23 ... 29 31 31]
 [22 22 23 ... 29 31 32]
 ...
 [24 23 21 ... 23 23 24]
 [25 23 22 ... 23 24 24]
 [24 23 23 ... 23 24 24]]
[[23 22 22 ... 24 25 24]
 [23 23 23 ... 24 24 26]
 [23 25 25 ... 22 23 24]
 ...
 [21 22 23 ... 24 25 24]
 [19 20 21 ... 24 26 27]
 [19 20 22 ... 25 25 26]]
[[20 21 22 ... 23 22 22]
 [20 20 20 ... 22 23 23]
 [19 19 19 ... 24 26 27]
 ...
 [18 17 14 ... 16 15 14]
 [15 13 12 ... 15 16 14]
 [15 14 13 ... 14 15 12]]
[[18 16 16 ... 14 14 13]
 [16 15 16 ... 13 14 13]
 [16 14 14 ... 13 14 15]
 ...
 [16 15 15 ...  3  3  3]
 [14 14 14 ...  3  3  3]
 [12 14 17 ...  3  3  3]]
[[12 17 18 ...  3  3  3]
 [14 17 16 ...  3 

In [27]:
import shutil

# specify the directory containing the subdirectories
new_index = 0
# loop through all subdirectories
for subdir in os.listdir(NEW_DIR):
    # construct the full path to the subdirectory
    subdir_path = os.path.join(NEW_DIR, subdir)
    
    # check if the item is a directory
    if os.path.isdir(subdir_path):
        # loop through all items in the subdirectory
        for item in os.listdir(subdir_path):
            # construct the full path to the item
            new_n = os.path.join(subdir_path, f"{new_index}.tif")
            os.rename(os.path.join(subdir_path, item), new_n)
            item_path = os.path.join(subdir, new_n)

            # move the item to the original directory
            shutil.move(item_path, NEW_DIR)
            new_index += 1

Error: Destination path '/media/quothbonney/SRTM/512/0.tif' already exists